In [1]:
import os
import cv2
import numpy as np
from keras.models import load_model

model_path = 'Mask.model'

if os.path.exists(model_path):
    
    model = load_model(model_path)
    print("Model loaded successfully.")

else:
    print(f"No model directory found at {model_path}")


Model loaded successfully.


In [2]:
cap = cv2.VideoCapture(0)

color_dict = {0: (0, 255, 0), 1: (0, 0, 255)}

while True:
    ret, frame = cap.read()
    
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        clsfr = cv2.CascadeClassifier(r"C:\Users\aashutosh kumar\Downloads\haarcascade_frontalface_default.xml")

        faces = clsfr.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3)
        
        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            
            resized = cv2.resize(face_img, (150, 150))

            if len(resized.shape) == 2:  # Grayscale image
                resized = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)
            elif resized.shape[2] == 1:  # Single-channel image
                resized = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)
            normalized = resized / 255.0
            
        
            reshaped = np.reshape(normalized, (1, 150, 150, 3))
            
            # Perform prediction using the model
            result = model.predict(reshaped)
            
            # Get the predicted label based on confidence threshold
            confidence = result[0][0]
            label = 0 if confidence >= 0.5 else 1
            label_text = "MASK" if confidence < 0.5 else "NO MASK"

            # Draw rectangle around the face region
            cv2.rectangle(frame, (x, y), (x+w, y+h), color_dict[label], 2)
            
            # Draw label text on the frame
            cv2.rectangle(frame, (x, y-40), (x+w, y), color_dict[label], -1)
            cv2.putText(frame, label_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        cv2.imshow('LIVE', frame)
        
       
        if cv2.waitKey(25) & 0xFF == ord('p'):
            break
    else:
        break

# Release the video capture device and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 25ms/step
